In [1]:
import asyncio
from time import sleep
from gpiozero import DigitalOutputDevice
from datetime import datetime
from dataclasses import dataclass, asdict
import multiprocessing
from queue import Empty
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

In [2]:
try:
    from max31855 import MAX31855
except ImportError:
    print("Stubbing MAX31855 & Power")
    class MAX31855:
        stubbed = True
        temp = 0
        def getTemperature(self):
            self.temp += 1
            return self.temp

    class DigitalOutputDevice:
        stubbed = True
        def __init__(self, pin, initial_value):
            self.value = initial_value
        def on(self):
            self.value = True
        def off(self):
            self.value = False

Stubbing MAX31855 & Power


In [3]:
class SimpleQueue:
    def __init__(self, depth):
        self.depth = depth
        self.queue = []

    def __getitem__(self, key):
        try:
            return self.queue[key]
        except:
            return None

    def __call__(self, value):
        self.queue.insert(0, value)
        while len(self.queue) > self.depth:
            self.queue.pop()

In [4]:
@dataclass
class KilnObservation:
    timestamp: str
    cycle: int
    temperature: float
    target: float
    power: float
    powerOn: bool
    setPower: float
    haltPowerChange: int
    powerStep: float
    slope: float
    accel: float
    cyclesUntilTarget: float
    error: float
    percentError: float
    realCycleTime: float
    lastTemperature: float

    def __repr__(self):
        reps = []
        for key, value in asdict(self).items():
            length = "10" if type(value) in [str, int, bool] else "10.8"
            reps.append("{key}: {value:<{length}}".format(key=key, value=value, length=length))
        return "Kiln=" + " | ".join(reps)

In [5]:
class KilnController:
    def __init__(self, targetQueue=None, stateQueue=None):
        self.maxPower = 70

        self.slopeBoost = 0.01
        self.accelBoost = 0.03
        
        self.cycleCount = 0
        self.cycleTime = 0.1
        self.realCycleTime = 1

        self.targetQueue = targetQueue
        try:
            self.targetTemp = self.targetQueue.get(block=False)
        except Empty:
            self.targetTemp = 0

        self.tempProbe = MAX31855()
    
        self.tempMemoryDepth = 60000
        self.tempMemory = SimpleQueue(self.tempMemoryDepth)
        self.tempMemory(self.temperature)
        
        self.setPower = 0
        self.setPowerMemory = SimpleQueue(60000)
        self.currentPower = 0
        self.powerWindow = 0.5

        self.power = DigitalOutputDevice(pin="GPIO17", initial_value=False)
        self.powerMemoryDepth = 60000
        self.powerOnMemory = SimpleQueue(self.powerMemoryDepth)
        self.powerMemory = SimpleQueue(self.powerMemoryDepth)
        
        self.restrictPowerChangeUntil = 0
        
        self.extendObservations(120)
        
        self.targetQueue = targetQueue
        self.stateQueue = stateQueue
    
    @property
    def filteredTemp(self):
        window = 10
        fT = []
        for i in range(0, window ** 2, window):
            selected = sorted(self.tempMemory.queue[i:i + window])[int(window / 2)]
            fT.extend([selected for i in range(window)])
        return fT[:self.tempMemoryDepth]
    
    @property
    def temperature(self):
        return self.tempProbe.getTemperature()
    
    def getCurrentPower(self):
        return (sum(self.powerOnMemory.queue) / len(self.powerOnMemory.queue)) * 100

    def observe(self):
        self.setPowerMemory(self.setPower)
        self.powerMemory(self.getCurrentPower())
        self.powerOnMemory(self.power.value)
        self.tempMemory(self.temperature)
    
    def extendObservations(self, n):
        self.setPowerMemory.queue = [self.setPower for i in range(n)] + self.setPowerMemory.queue
        self.powerOnMemory.queue = [self.power.value for i in range(n)] + self.powerOnMemory.queue
        
        pMValue = self.getCurrentPower()
        self.powerMemory.queue = [pMValue for i in range(n)] + self.powerMemory.queue
        
        currentTemp = self.temperature
        self.tempMemory.queue = [currentTemp for i in range(n)] + self.tempMemory.queue

    def processPower(self):
        try:
            local = self.powerOnMemory[:10]
            tooMuchEnergy = (sum(local) / 10) * 100 > self.maxPower
            wantPower = self.getCurrentPower() < min(self.setPower, self.maxPower)
            
            if tooMuchEnergy:
                print("Too Much Energy")
                self.power.off()
            elif wantPower:
                print("Energizing")
                self.power.on()
            else:
                print("Depowering")
                self.power.off()
        except:
            print("ERROR!")
            self.power.off()
            raise
    
    def setTargetTemperature(self, targetTemperature):
        print(f"Setting target temperature to: {targetTemperature}")
        self.targetTemp = min(int(targetTemperature), 2300)
        self.restrictPowerChangeUntil = 0
        
    def characterizeKiln(self):
        start = 0
        stop = 120
        obs = self.filteredTemp[start:stop]
        
        currentWindow = obs[:int(stop / 2)]
        secondWindow = obs[int(stop / 3):]

        temperature = currentWindow[0]
        error = self.targetTemp - temperature
        percentError = (error / (self.targetTemp if self.targetTemp != 0 else 1/10000)) * 100
        slope = (currentWindow[0] - currentWindow[-1]) / (len(currentWindow) * self.realCycleTime)
        secondWindowSlope = (secondWindow[0] - secondWindow[-1]) / (len(secondWindow) * self.realCycleTime)
        accel = (slope - secondWindowSlope) / 2
        if slope == 0 or (error * slope) < 0:
            cyclesUntilTarget = 99999
        else:
            cyclesUntilTarget = min(int(abs(error) / slope + 0.5), 99999)

            
        powerStep = min(max(25 * (error / 1500), 0.1), 5)

        state = KilnObservation(**{
            "timestamp": str(datetime.utcnow()),
            "cycle": self.cycleCount,
            "temperature": obs[0],
            "target": self.targetTemp,
            "power": self.getCurrentPower(),
            "powerOn": self.power.value,
            "setPower": self.setPower,
            "haltPowerChange": self.restrictPowerChangeUntil,
            "powerStep": powerStep,
            "slope": slope,
            "accel": accel,
            "cyclesUntilTarget": cyclesUntilTarget,
            "error": error,
            "percentError": percentError,
            "realCycleTime": self.realCycleTime,
            "lastTemperature": currentWindow[-1]
        })
        return state

    def tune(self):
        if self.targetTemp < 80:
            print("Kiln Disabled")
            self.setPower = 0
            return

        kilnChar = self.characterizeKiln()
        print(f"Tuning: {kilnChar}")
        
        newPower = self.setPower
        if kilnChar.percentError < 0.01:
            print("Hard Power Cutoff")
            newPower = 0
        elif (kilnChar.error > 0 and kilnChar.cyclesUntilTarget < 15):
            print("Preemptively Cutting Power")
            newPower = 0
        elif (
            self.cycleCount > self.restrictPowerChangeUntil and 
            (kilnChar.slope <= self.slopeBoost or kilnChar.accel <= self.accelBoost)
        ):
            print("Adding Energy")
            newPower = max(kilnChar.power + kilnChar.powerStep, self.setPower + kilnChar.powerStep)
            self.restrictPowerChangeUntil = self.cycleCount + 15

        if self.setPower != newPower:
            self.setPower = max(min(newPower, self.maxPower), 0)
            print(f"Tuned power to {self.setPower}")
        else:
            print("Holding power constant")
    
    def reportState(self, state=None):
        if self.stateQueue is None:
            return
        if state is None:
            state = self.characterizeKiln()
        self.stateQueue.put(state)
    
    def __repr__(self):
        return f"Kiln={self.characterizeKiln()}"

    def cycle(self):
        start = datetime.utcnow()
        if self.targetQueue is not None:
            try:
                self.setTargetTemperature(self.targetQueue.get(block=False))
            except Empty:
                pass

        self.cycleCount += 1
        self.observe()
        self.processPower()
        if len(self.tempMemory.queue) > 120:
            state = self.tune()
            self.reportState(state)
            self.realCycleTime = (datetime.utcnow() - start).total_seconds()
    
    def run(self):
        while True:
            self.cycle()
            sleep(self.cycleTime)

In [6]:
class KilnDrone:
    def __init__(self):
        self.targetQueue = multiprocessing.Queue()
        self.stateQueue = multiprocessing.Queue()
        self.states = SimpleQueue(60000)
        self.controller = KilnController(targetQueue=self.targetQueue, stateQueue=self.stateQueue)
        self.cyclesBetweenRenders = 150
        self.renderWindow = 50
    
    def setTargetTemperature(self, temperature):
        assert type(float(temperature)) == float
        self.targetQueue.put(temperature)  

    async def monitor(self, render=False):
        state = self.stateQueue.get()
        self.states(state)
        if render:
            if state.cycle > self.renderWindow:
                self.renderWindow = state.cycle + self.cyclesBetweenRenders
                self.renderKilnDrone(displayFigure=True)
            elif state.cycle % 10 == 0:
                print(f"Observer will start rendering in {self.renderWindow - state.cycle} cycles")

    def renderKilnDrone(self, cyclesAgo=6000, renderPowerImpluse=True, displayFigure=False):
        print("Observer Rendering KilnDrone")
        temperature = [s.temperature for s in self.states[:cyclesAgo]]
        targetTemperature = [s.target for s in self.states[:cyclesAgo]]
        setPower = [s.setPower for s in self.states[:cyclesAgo]]
        power = [s.power for s in self.states[:cyclesAgo]]
        powerImpulse = [s.powerOn * s.setPower for s in self.states[:cyclesAgo]]

        fig, ax = plt.subplots()
        ax.set_xlabel("Cycles Ago")
        ax2 = ax.twinx()
        ax.set_ylabel("Temperature (*F)")

        ln1 = ax.plot(temperature, color="green", label="Temperature", zorder=8)
        ln2 = ax.plot(targetTemperature, color="blue", label="Target Temperature", zorder=7)

        if renderPowerImpluse:
            ln3 = ax2.plot(powerImpulse, color="red", label="Power Impulse", zorder=4)

        ln4 = ax2.plot(setPower, color="darkred", label="Set Power", zorder=5)
        ln5 = ax2.plot(power, color="turquoise", label="Power", zorder=6)

        ax2.set_ylabel("Power Duty Cycle")
        lines = ln1 + ln2 + ln4 + ln5
        if renderPowerImpluse:
            lines = lines + ln3
        labels = [l.get_label() for l in lines]
        ax.legend(lines, labels, loc="upper right").set_zorder(100)
        plt.title("Temperature and Power over Time")
        ax.set_zorder(ax2.get_zorder() + 1)
        ax.patch.set_visible(False)
        if displayFigure:
            display(plt.show())
        return fig
    
    def multiProcRun(self, target=None):
        if target is not None:
            self.setTargetTemperature(target)
        
        controllerProc = multiprocessing.Process(target=self.controller.run)
        monitorProc = multiprocessing.Process(target=self.monitor)
        
        try:
            print("Starting kiln processes")
            controllerProc.start()
            monitorProc.start()
            controllerProc.join()
        except:
            print("Killing processes")
            controllerProc.terminate()
            monitorProc.terminate()
        finally:
            print("Setting heater power to off")
            self.controller.power.off()
    
    async def asyncRun(self):
        while True:
            await asyncio.sleep(self.controller.cycleTime)
            self.controller.cycle()
            await self.monitor()